In [18]:
import os, time,re,json
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options as ChromeOptions

def scrape_html(id):
   
    path = f"https://www.shorehamvehicleauctions.com/auction/{id}"
    folder = "html"
    os.makedirs(folder, exist_ok=True)

    # Selenium setup
    options = ChromeOptions()
    options.headless = True
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get(path)
    driver.maximize_window()

    # Accept cookies
    try:
        cookie = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, './/div[@class = "cc-compliance"]')))
        cookie.click()
    except:
        pass

    # Login
    try:
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/a[text()="Login"]'))).click()
        WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.ID,'username'))).send_keys("fourbrotherstrading@icloud.com")
        pw = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.ID,'password')))
        pw.send_keys("Muhssan7865@")
        WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, './/button[text() = "Sign in"]'))).click()
    except:
        print("Login issue")

    time.sleep(2)  # wait for login

    # Selenium cookies to requests
    session = requests.Session()
    for c in driver.get_cookies():
        session.cookies.set(c['name'], c['value'])
        
    
    raw_text = WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "span.select2-selection__choice__display"))
    ).text.strip()

    auction_name = re.sub(r'^.*\d{2}:\d{2}\s*', '', raw_text).strip()

    # Save auction_name to JSON
    if auction_name:  # Only save if we actually got a name
        folderDatabase = "database"
        os.makedirs(folderDatabase, exist_ok=True)

        json_file = os.path.join(folderDatabase, "data.json")
        data = {"auction_name": auction_name}

        with open(json_file, "w", encoding="utf-8") as f:
            json.dump(data, f, indent=4, ensure_ascii=False)

        print(f"✅ Auction name saved to {json_file}")
    else:
        print("⚠ No auction name found")


    

    # Total cars
    try:
        cars_text = WebDriverWait(driver, 7).until(EC.presence_of_element_located((By.XPATH, './/span[@class="info-results"]'))).text.strip()
        total_cars = int(cars_text.split(" ")[-2])
        print(f"Total cars: {total_cars}")
    except:
        print("No car count found")
        total_cars = 999999

    count = 0
    while count < total_cars:
        try:
            cars_links = WebDriverWait(driver, 5).until(
                EC.presence_of_all_elements_located((By.XPATH, './/a[@class="btn btn-secondary btn-icon btn-icon-arrow btn-icon-right"]'))
            )

            for i in range(len(cars_links)):
                if count >= total_cars:
                    break
                
         

                cars_links = WebDriverWait(driver, 5).until(
                    EC.presence_of_all_elements_located((By.XPATH, './/a[@class="btn btn-secondary btn-icon btn-icon-arrow btn-icon-right"]'))
                )
                driver.execute_script("arguments[0].scrollIntoView();", cars_links[i])
                time.sleep(0.5)
                cars_links[i].click()
                time.sleep(1)

      
                try:
                    reg = WebDriverWait(driver, 3).until(
                        EC.presence_of_element_located((By.XPATH, './/li[span[text()="Registration"]]/span[@class="item-value"]'))
                    ).text.strip()
                except:
                    reg = f"reg_not_found_{count+1}"

                if re.match(r'^\d{2}/\d{2}/\d{2,4}$', reg):
                    print(f"⏭ Skipping date-format REG → {reg}")
                    driver.back()
                    time.sleep(0.5)
                    continue


                car_url = driver.current_url
                try:
                    html = session.get(car_url, timeout=5).text
                except:
                    print("⚠ Request session failed, using Selenium page source.")
                    html = driver.page_source

                # Save HTML
                file_name = f"{folder}/{reg}.html"
                with open(file_name, "w", encoding="utf-8") as f:
                    f.write(html)
                print(f"Saved ➤ {file_name}")

                count += 1
                driver.back()
                time.sleep(0.5)

            # Next page
            try:
                next_page = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, '(.//a[@title = "next"])[2]')))
                driver.execute_script("arguments[0].scrollIntoView();", next_page)
                time.sleep(0.5)
                next_page.click()
            except:
                print("⚠ No more pages.")
                break

        except:
            break

    driver.quit()

# Run
id = 142
scrape_html(id)



✅ Auction name saved to database\data.json
Total cars: 110
⏭ Skipping date-format REG → 25/11/3770
Saved ➤ html/PL-8176.html
Saved ➤ html/MF19SKV.html
Saved ➤ html/DV72FHG.html
Saved ➤ html/FL20YXZ.html
Saved ➤ html/HT21XFE.html
Saved ➤ html/MJ23BLX.html
Saved ➤ html/DL72OFX.html
Saved ➤ html/LF72MKR.html
Saved ➤ html/DS72OUD.html
Saved ➤ html/KT23WNZ.html
Saved ➤ html/WM72TWE.html
Saved ➤ html/HS24GBE.html
Saved ➤ html/WP72UVB.html
Saved ➤ html/FL22WEO.html
Saved ➤ html/WR22HBL.html
Saved ➤ html/KR71FVM.html
Saved ➤ html/LA72HUZ.html
Saved ➤ html/WG23DVW.html
Saved ➤ html/CK71AXM.html
Saved ➤ html/NV22ORZ.html
Saved ➤ html/WN22EYR.html
Saved ➤ html/FL22WCV.html
Saved ➤ html/WN74CZE.html
Saved ➤ html/DV72BWY.html
Saved ➤ html/LB74NFA.html
Saved ➤ html/KN71DHZ.html
Saved ➤ html/LG72YPK.html
Saved ➤ html/MT72WSE.html
Saved ➤ html/HK71ARF.html
Saved ➤ html/FL22XUS.html
Saved ➤ html/WR72CDK.html
Saved ➤ html/FA21NBZ.html
Saved ➤ html/KR22MLN.html
Saved ➤ html/HV71VHU.html
Saved ➤ html/NV72

In [19]:
from bs4 import BeautifulSoup
import pandas as pd
import os,re,json

def findtag(soup, key):
    li = soup.find("li", class_="detail-item", text=lambda t: t and key in t)
    if li:
        val_tag = li.find("span", class_="item-value")
        if val_tag:
            return val_tag.get_text(strip=True)

    detail_list = soup.find("ul", class_="detail-list")
    if detail_list:
        for li in detail_list.find_all("li", class_="detail-item"):
            key_tag = li.find("span", class_="item-key")
            val_tag = li.find("span", class_="item-value")
            if key_tag and val_tag and key_tag.get_text(strip=True) == key:
                return val_tag.get_text(strip=True)
    return ""

def html_to_csv(folder="html",auctionname="Shoreham Vehicle Auctions"):
    records = []

    for file in os.listdir(folder):
        if file.endswith(".html"):
            path = os.path.join(folder, file)
            with open(path, "r", encoding="utf-8") as f:
                soup = BeautifulSoup(f, "html.parser")

            data = {}

            
            page_title = soup.find("div", class_="page-title")
            if page_title:
                next_h1 = page_title.find_next("h1")
                data["Title"] = next_h1.get_text(strip=True) if next_h1 else ""
                doors = ""
                if data["Title"]:
                    title = data["Title"]
                    match = re.search(r'(\d+)\s*dr', title, re.IGNORECASE)
                    doors = match.group(1) if match else ""
                
                data["Doors"] = doors
            else:
                h1_tags = soup.find_all("h1")
                data["Title"] = h1_tags[1].get_text(strip=True) if len(h1_tags) > 1 else ""

            data["Auction Name"] = auctionname
            data["Center"] = "Lancing"
            data["Reg"] = file.replace(".html", "")

  
            try:
                lot_tag = soup.find("h2", class_="h3 detail-lot")
                if lot_tag:
                    text = lot_tag.get_text(strip=True)
                    lot_match = re.search(r"Lot\s*(\d+|TBC)", text)
                    date_match = re.search(r"\d{2}/\d{2}/\d{4}", text)
                    time_match = re.search(r"\d{1,2}:\d{2}", text)
                    data["Lot"] = lot_match.group(1) if lot_match else "na"
                    data["Start Date"] = date_match.group(0) if date_match else "na"
                    data["Start Time"] = time_match.group(0) if time_match else "na"
                else:
                    data["Lot"] = data["Start Date"] = data["Start Time"] = "na"
            except:
                data["Lot"] = data["Start Date"] = data["Start Time"] = "na"

            data["Make"] = findtag(soup, "Make")
            data["Model"] = findtag(soup, "Model")
            data["Variant"] = findtag(soup, "Variant")
            data["Former Keepers"] = findtag(soup, "Former Keepers")
            data["Fuel Type"] = findtag(soup, "Fuel")
            data["Transmission"] = findtag(soup, "Transmission")
            data["Colour"] = findtag(soup, "Colour")
            data["VAT Status"] = findtag(soup, "VAT")
            data["MOT Expiry Date"] = findtag(soup, "MOT")
            data["V5"] = findtag(soup, "V5")
            data["Keys"] = findtag(soup, "Keys")
            data["Service History"] = findtag(soup, "Service History")
            data["CAP Clean"] = findtag(soup, "CAP Clean")
            data["CAP Average"] = findtag(soup, "CAP Average")
            data["CAP Below"] = findtag(soup, "CAP Below")
            data["Vendor"] = findtag(soup, "Vendor")
            data["Last service mileage"] = findtag(soup, "Last Service Miles")
            data["Last Service"] = findtag(soup, "Last Service Date")
            data["No of services"] = findtag(soup, "Number of Stamps")
            
            registered_date = findtag(soup, "Registered")
            data["D.O.R"] = registered_date
            if registered_date:
                year = registered_date.split("/")[-1]
                data["Year"] = year
            else:
                data["Year"] = "na"
            cc_text = findtag(soup, "CC")  
            if cc_text and cc_text.strip():  
                try:
                    cc_value = int(cc_text.replace(',', '').strip())
                    data["CC"] = round(cc_value / 1000, 1) 
                except ValueError:
                    data["CC"] = "na"
            else:
                data["CC"] = "na"

            miles_text = findtag(soup, "Miles") 
            if miles_text != "na":
                mileage_match = re.search(r'[\d,]+', miles_text)
                if mileage_match:
                    data["Mileage"] = int(mileage_match.group(0).replace(',', ''))
                else:
                    data["Mileage"] = "na"
                if "Not Warranted" in miles_text:
                    data["Mileage Warranted"] = "No"
                elif "Warranted" in miles_text:
                    data["Mileage Warranted"] = "Yes"
                else:
                    data["Mileage Warranted"] = "na"
            else:
                data["Mileage"] = "na"
                data["Mileage Warranted"] = "na"

            mechanical = {}
            interior = {}


            for alert in soup.find_all("div", class_="alert alert-success"):
                key_tag = alert.find("span", style="font-size: 18px")
                val_tag = alert.find("span", class_="float-right")
                if key_tag and val_tag:
                    key = key_tag.get_text(strip=True)
                    val = val_tag.get_text(strip=True)
                    mechanical[key] = val


            interior_div = soup.find("strong", text="Interior")
            if interior_div:
                table = interior_div.find_next("table")
                if table:
                    rows = table.find_all("tr")
                    for row in rows:
                        cols = row.find_all("td")
                        if len(cols) == 2:
                            key = cols[0].get_text(strip=True)
                            val = cols[1].get_text(strip=True)
                            interior[key] = val


            data["General Condition"] = json.dumps({
                "Mechanical": mechanical,
                "Interior": interior
            }, indent=4)


            engine_status = mechanical.get("Engine runs", "na")
            data["Non Runner"] = "No" if engine_status == "OK" else "Yes"

            notes_header = soup.find("h2", class_="h3", text=lambda t: t and "Notes" in t)
            notes_content = {}
            if notes_header:
                p_tag = notes_header.find_next("p")
                if p_tag:
                    # Split by <br> or newline
                    lines = [line.strip() for line in p_tag.decode_contents().split("<br>") if line.strip()]
                    for i, line in enumerate(lines, 1):
                        notes_content[f"Note {i}"] = line

            # Add to Additional Information
            data["Additional Information"] = json.dumps({
                "Notes": notes_content
            }, indent=4)
            
            tyres_div = soup.find("strong", text="Tyres")
            tyres_condition = ""
            if tyres_div:
                table = tyres_div.find_next("table")
                if table:
                    rows = table.find_all("tr")
                    tyre_values = []
                    for row in rows:
                        cols = row.find_all("td")
                        if len(cols) >= 3:
                            key = cols[0].get_text(strip=True)
                            val = cols[2].get_text(strip=True)
                            tyre_values.append(f"{key}: {val if val else 'na'}")
                    tyres_condition = ", ".join(tyre_values)

            data["Tyres Condition"] = tyres_condition
            
            grade_div = soup.find("div", class_=re.compile(r"single-grade"))
            grade_number = None
            
            pdfTag= soup.find("a", href=re.compile(r"\.pdf$"))
            if pdfTag and pdfTag.get("href"):
                data["Inspection Report"]=f"https://www.shorehamvehicleauctions.com{pdfTag.get("href")}"                

            if grade_div:
                img = grade_div.find("img")
                if img and img.get("src"):
                    match = re.search(r"nama-(\d+)-mobile\.svg", img["src"])
                    if match:
                        grade_number = match.group(1)

            data["Grade"]=grade_number
            
            
            damage_div = soup.find("div", id="damagegallery")
            damage_images = []
            damage_details = []

            if damage_div:
                for img in damage_div.find_all("img"):
                    img_url = img.get("data-image") 
                    description = img.get("data-description")  
                    if img_url:
                        damage_images.append(img_url)
                    if description:
                        damage_details.append(description)

            data["Damage Images"] = ",".join(damage_images)
            data["Damage Details"] = ",".join(damage_details)
            
            
            images = []

           
            for img in soup.select("div.gallery-main img"):
                src = img.get("data-image") 
                if src:
                    
                    images.append(src)

            data["Images"] = ", ".join(images)

            
            
            records.append(data)


    df = pd.DataFrame(records)
    df.to_csv("shoreham_data.csv", index=False)
    print("Saved CSV: shoreham.csv")


json_file = os.path.join("database", "data.json")

auction_name = "Shoreham Vehicle Auctions"

if os.path.exists(json_file):
    with open(json_file, "r", encoding="utf-8") as f:
        try:
            data = json.load(f)
            if "auction_name" in data and data["auction_name"].strip():
                auction_name = data["auction_name"].strip()
        except json.JSONDecodeError:
            print("⚠ JSON file is empty or corrupted, using default name.")

html_to_csv(folder="html", auctionname=auction_name)


C:\Users\HP\AppData\Local\Temp\ipykernel_19276\1430538366.py:6: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  li = soup.find("li", class_="detail-item", text=lambda t: t and key in t)
C:\Users\HP\AppData\Local\Temp\ipykernel_19276\1430538366.py:135: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  interior_div = soup.find("strong", text="Interior")
C:\Users\HP\AppData\Local\Temp\ipykernel_19276\1430538366.py:157: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  notes_header = soup.find("h2", class_="h3", text=lambda t: t and "Notes" in t)
C:\Users\HP\AppData\Local\Temp\ipykernel_19276\1430538366.py:172: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  tyres_div = soup.find("strong", text="Tyres")


Saved CSV: shoreham.csv


In [ ]:
from urllib.parse import urlparse, urljoin
import threading, requests, os, re
import pandas as pd
from PIL import Image, ImageDraw, ImageFont


df = pd.read_csv("shoreham_data.csv")


reg_img = df[['Reg', "Images", "Damage Images"]]
reports = df[['Reg', "Inspection Report"]]


def add_watermark_to_image(image_path, text="Sourced from Shoreham Vehicle Auctions"):
    try:
        image = Image.open(image_path).convert("RGBA")
        txt_layer = Image.new("RGBA", image.size, (255, 255, 255, 0))
        draw = ImageDraw.Draw(txt_layer)

        try:
            font = ImageFont.truetype("arial.ttf", 20)
        except:
            font = ImageFont.load_default()

        margin = 10
        bbox = draw.textbbox((0, 0), text, font=font)
        tw, th = bbox[2] - bbox[0], bbox[3] - bbox[1]
        x, y = image.width - tw - margin, image.height - th - margin

        draw.rectangle([x - margin, y - margin, x + tw + margin, y + th + margin], fill=(0,0,0,160))
        draw.text((x, y), text, font=font, fill=(255,255,255,200))
        watermarked = Image.alpha_composite(image, txt_layer).convert("RGB")
        watermarked.save(image_path)
        print(f"✔ Watermarked: {image_path}")

    except Exception as e:
        print(f"⚠ Watermark Error: {e}")


def download_images(data, main_folder="Images"):
    os.makedirs(main_folder, exist_ok=True)

    for index, row in data.iterrows():
        reg_no = str(row["Reg"]).strip()

    
        img_urls = [u for u in re.split(r',\s*', str(row["Images"])) if u]
        dmg_urls = [u for u in re.split(r',\s*', str(row["Damage Images"])) if u]

        reg_folder = os.path.join(main_folder, reg_no)
        original_folder = os.path.join(reg_folder, "Original")
        damage_folder = os.path.join(reg_folder, "Damage")

        os.makedirs(original_folder, exist_ok=True)
        os.makedirs(damage_folder, exist_ok=True)

        def save_img(url, folder, idx):
            url = url.strip()
            if not url:
                return

            if not url.startswith(("http://", "https://")):
                url = urljoin("https://", url)

            parsed = urlparse(url)
            if not parsed.netloc:
                print(f"❌ Invalid URL Skipped: {url}")
                return

            full_path = os.path.join(folder, f"{reg_no}_{idx}.jpg")


            if os.path.exists(full_path):
                print(f"⏩ Skipped (Exists): {full_path}")
                return

            try:
                response = requests.get(url, stream=True, timeout=20)
                response.raise_for_status()

                with open(full_path, "wb") as f:
                    for chunk in response.iter_content(1024):
                        f.write(chunk)

                add_watermark_to_image(full_path)
                print(f"📌 Saved: {full_path}")

            except Exception as e:
                print(f"⚠ Error downloading: {url} -> {e}")

        for i, url in enumerate(img_urls):
            save_img(url, original_folder, i+1)


        for i, url in enumerate(dmg_urls):
            save_img(url, damage_folder, i+1)


def download_reports(data, main_folder="Reports"):
    os.makedirs(main_folder, exist_ok=True)

    report_folder = os.path.join(main_folder, "Inspection Reports")
    os.makedirs(report_folder, exist_ok=True)

    for i, row in data.iterrows():
        reg_no = str(row["Reg"]).strip()
        report_url = row["Inspection Report"]

        if not report_url or pd.isna(report_url):
            print(f"⚠ Missing Report for {reg_no}")
            continue

        if not report_url.startswith(("http://","https://")):
            report_url = urljoin("https://", report_url)

        file_path = os.path.join(report_folder, f"{reg_no}.pdf")


        if os.path.exists(file_path):
            print(f"⏩ Skipped (Exists): {file_path}")
            continue

        try:
            response = requests.get(report_url, timeout=20)
            response.raise_for_status()

            with open(file_path, "wb") as f:
                f.write(response.content)

            print(f"📄 Report Downloaded: {file_path}")

        except Exception as e:
            print(f"❌ Failed: {report_url} -> {e}")


def start_funcs():
    t1 = threading.Thread(target=download_images, args=(reg_img,))
    t2 = threading.Thread(target=download_reports, args=(reports,))

    t1.start(); t2.start()
    t1.join(); t2.join()
 
if __name__ == "__main__":
    start_funcs()


📄 Report Downloaded: Reports\Inspection Reports\BF63DMU.pdf
📄 Report Downloaded: Reports\Inspection Reports\BF63DPE.pdf
✔ Watermarked: Images\BF63DMU\Original\BF63DMU_1.jpg
📌 Saved: Images\BF63DMU\Original\BF63DMU_1.jpg
📄 Report Downloaded: Reports\Inspection Reports\BF63DTU.pdf
📄 Report Downloaded: Reports\Inspection Reports\BF73YMW.pdf
✔ Watermarked: Images\BF63DMU\Original\BF63DMU_2.jpg
📌 Saved: Images\BF63DMU\Original\BF63DMU_2.jpg
📄 Report Downloaded: Reports\Inspection Reports\BN74ZRL.pdf
📄 Report Downloaded: Reports\Inspection Reports\BU63BWO.pdf
📄 Report Downloaded: Reports\Inspection Reports\BX63VSC.pdf
✔ Watermarked: Images\BF63DMU\Original\BF63DMU_3.jpg
📌 Saved: Images\BF63DMU\Original\BF63DMU_3.jpg
📄 Report Downloaded: Reports\Inspection Reports\BX63VTW.pdf
❌ Failed: https://www.shorehamvehicleauctions.com/motorvehicledamageimage?key=StorageS3::privileged/motorvehicle/13849/1763554734_Q7L3re.pdf -> 404 Client Error: Not Found for url: https://www.shorehamvehicleauctions.com